In [2]:
%pylab inline
import scipy.stats as sps
import scipy.integrate as spi
import seaborn as sns
import csv

import dask
from dask.distributed import Client

import time

from dask_jobqueue import SLURMCluster

sns.set_context('talk')

Populating the interactive namespace from numpy and matplotlib


In [ ]:
run_local = False
n_workers_init = 6

if run_local:
    client = Client()
    client.cluster.scale(n_workers_init)
    time.sleep(0.5) # allow client time to respond to scaling before display
else:
    # note the specifed walltime, don't use too much or too little, 01:30:00 is a good baseline, 
    # you just need enough time to finish 'gathering' to props_all before the jobs die
    # you can always spin up more jobs later
    # you will launch many jobs, so you don't need multiple processes, a lot of ram or multiple threads
#     cluster = SLURMCluster(queue="short",walltime='01:30:00',job_cpu=1,job_mem='6G',threads=1,processes=1,memory='6GB')
    
    #SHORT RUNS
#     cluster = SLURMCluster(queue="short",walltime='01:30:00',job_cpu=1,job_mem='6G',cores=1,processes=1,memory='6G')
    cluster = SLURMCluster(queue="short",walltime='01:30:00',job_cpu=1,job_mem='6G',cores=1,processes=1,memory='6G')
    cluster.start_workers(n_workers_init)
    client = Client(cluster)

client

In [ ]:
def plot_max_lines(MTs, STs, MN, mTa, exp_key = None):
    #Indices of E(T_s) values for max fitness as a function of std
    max_MTs_vs_std = np.argmax(MN, axis=0)
    #Indices of std(T_s) values for max fitness as a function of E(T_s)
    max_STs_vs_mean = np.argmax(MN, axis=1)
    
    fig, axs = plt.subplots(1,2, figsize=(7,3.5), tight_layout=True)

    axs[0].plot(STs, MTs[max_MTs_vs_std], 'o-')
    axs[0].set_xlabel('$\sigma(T_s)$')
    axs[0].set_ylabel('$E(T_s)$ for max. fitness')

    axs[1].plot(MTs, [STs[e] for e in max_STs_vs_mean], 'o-')
    axs[1].set_xlabel('$E(T_s)$')
    axs[1].set_ylabel('$\sigma(T_s)$ for max. fitness')
    
    if exp_key:
        plt.savefig('./exp_'+exp_key+'_maxs.png')
    
def save_experiment(exp_key, res, kwargs, kwargs_s):

    # Save NN
    np.save('./iexp_'+exp_key, res)
    # Save metadata (parameters)
    file = open('./iexp_'+exp_key+'_meta.csv', "w")
    w = csv.writer(file)
    for key, val in kwargs.items():
        w.writerow([key, val])
    for key, val in kwargs_s.items():
        w.writerow([key, val])
    file.close()
    

def integrand_1(s, Ts):
    return np.exp(-s)*Ts.pdf(s)

def integral_1(ta, t, Ts):
    return spi.quad(integrand_1, ta, t, args=(Ts))[0]

def mN_det_Ta(mTa, t, Ts):
    # OJO: Only when t > ta
    return np.exp(t)*integral_1(mTa, t, Ts) + 1 - Ts.cdf(t)

def integrand_2(ta, t, Ta, Ts):
    return Ta.pdf(ta)*integral_1(ta, t, Ts)

def integral_2(t, Ta, Ts):
    return spi.quad(integrand_2, 0, t, args=(t, Ta, Ts))[0]

def mN_stoch_Ta(t, Ta, Ts):
    return np.exp(t)*integral_2(t, Ta, Ts)+1-Ts.cdf(t)

def run_deterministic_one(mTs, sTs, mTa, sTa=0, tg=8):
    
    if sTs != 0:
        Ts = sps.norm(mTs, sTs)
        return mN_det_Ta(mTa, tg, Ts)
    else:
        return 

def run_deterministic_single_sTs(sTs, MTs, mTa, sTa, tg=8):
    N = np.zeros(len(MTs))
    for i in range(len(MTs)):
        mTs = MTs[i]
        N[i] = run_deterministic_one(mTs, sTs, mTa, sTa=0, tg=8)
    return N

def run_deterministic(MTs, STs, mTa, sTa=0, tg=8):
    
    NN = np.zeros((len(MTs), len(STs)))
    
    for i in range(len(MTs)):
        mTs = MTs[i]
        for j in range(len(STs)):
            sTs = STs[j]
            
            NN[i,j] = run_deterministic_one(mTs, sTs, mTa, sTa=0, tg=tg)
#             Ts = sps.norm(mTs, sTs)
#             NN[i,j] = mN_det_Ta(mTa, tg, Ts)
    return NN

def run_stochastic_one(mTs, sTs, mTa, sTa, tg):
    
    Ts = sps.norm(mTs, sTs)
    Ta = sps.norm(mTa, sTa)
    return mN_stoch_Ta(tg, Ta, Ts)

def run_stochastic_single_sTs(sTs, MTs, mTa, sTa, tg):
    N = np.zeros(len(MTs))
    for i in range(len(MTs)):
        mTs = MTs[i]
        N[i] = run_stochastic_one(mTs, sTs, mTa, sTa, tg=8)
    return N

def run_stochastic(MTs, STs, Ta, tg):
    
    NN = np.zeros((len(MTs), len(STs)))
    
    for i in range(len(MTs)):
        mTs = MTs[i]
        for j in range(len(STs)):
            sTs = STs[j]
            Ts = sps.norm(mTs, sTs)
            NN[i,j] = mN_stoch_Ta(tg, Ta, Ts)
    return NN